In [39]:
import codecs
'''
预处理：
1. 人名/nr需要进行合并
2. 时间/t需要进行合并
3. 中括号进行合并
4. 全角统一转换成半角
'''

def strQ2B(ustring):
    """全角转半角"""
    rstring = ""
    for uchar in ustring:
        inside_code=ord(uchar)
        if inside_code == 12288:                              #全角空格直接转换            
            inside_code = 32 
        elif (inside_code >= 65281 and inside_code <= 65374): #全角字符（除空格）根据关系转化
            inside_code -= 65248
        rstring += chr(inside_code)
    return rstring

def sentence_process(line):
    line = line[:-2]
    state = 0
    state_key = ''
    concat_list = ''
    result_list = []

    for n, w in enumerate(line.split('  ')):
        if state == 2:
            #[香港/ns 特别/a 行政区/n]ns ==> 香港特别行政区/ns
            if ']' in w:
                result_list.append(concat_list + w.split('/')[0] + '/' + w.split(']')[-1])
                concat_list = ''
                state = 0
            else:
                concat_list += w.split('/')[0]
        elif state == 1:
            if w.split('/')[-1] == state_key:
                concat_list += w.split('/')[0]
            else:
                result_list.append(concat_list + '/' + state_key)
                result_list.append(w)
                concat_list = ''
                state = 0
        else:
            if w.startswith('['):
                concat_list = w[1:].split('/')[0]
                state = 2
            elif w.endswith('/nr') or w.endswith('/t'):
                state_key = w.split('/')[-1]
                concat_list = w.split('/')[0]
                state = 1
            else:
                result_list.append(w)
    return strQ2B('  '.join(result_list))

with codecs.open('199801-utf8-new.txt', 'w', 'utf-8') as wf:
    with codecs.open('199801-utf8.txt', 'r', 'utf-8') as f:
        for line in f.readlines():
            new_line = sentence_process(line)
            wf.write(new_line + '\n')
